# [이론2] 텍스트 표현 (Text Representation)

---

## 학습 목표
- 컴퓨터가 자연어를 이해하여 표현한 텍스트 표현에 대해 살펴봅시다.
- 어떠한 텍스트 표현이 있는지 살펴봅시다.

---

## 목차

### [이론2] 텍스트 표현 (Text Representation)
1. 텍스트 표현 소개
2. Word2vec
3. fastText

---

##  [이론2] 텍스트 표현 (Text Representation)

### 1. 텍스트 표현 소개

지난 시간에 우리는 자연어처리(Natural Language Processing, NLP)에 대해 간략히 살펴보았습니다.

자연어처리란 사람과 컴퓨터가 커뮤니케이션을 할 때 마치 사람과 사람끼리 커뮤니케이션을 하는 것처럼 자연어로 커뮤니케이션을 하도록 하는 일련의 작업을 뜻합니다. 

이 일련의 작업은 크게 아래 그림과 같이 볼 수 있습니다.

<img src="img/NLP.jpg" width="50%" height="50%" title="NLP" alt="NLP"></img>

위 그림은 사람과 컴퓨터가 자연어로 커뮤니케이션을 할 때의 모습입니다.

이번 시간에는 여기서 자연어 및 정보에 대한 컴퓨터만의 표현인 Representation에 대해 공부하겠습니다.

#### Representation

텍스트 표현에는 다양한 방법들이 있습니다. 이를 정리하면 아래와 같습니다.

- 개수 기반(Count-based)
  - 근처 단어들의 개수를 새어서 만드는 방법
  - 예: tf-idf, PPMI
- 클래스 기반(Class-based)
  - 계층적 클러스터링을 통해 만드는 방법
  - 예: Brown clusters
- 분산 추론 기반 임베딩(Distributed prediction-based embeddings)
  - 근처 단어 및 멀리 있는 단어들을 구분하는 분류기(classifier)를 학습시켜 만드는 방법
  - 예: Word2vec, fastText
- 분산 문맥 임베딩(Distributed contextual embeddings)
  - 언어 모델(language model)에서 나온 임베딩
  - 예: ELMo, BERT, GPT

기존에 많이들 사용되는 텍스트 표현은 개수 기반 혹은 클래스 기반 기법입니다. 이들은 간단하게 텍스트의 기본 토큰인 단어를 표현합니다.

하지만 최근 딥러닝의 발전으로 세 번째와 네 번째 기법이 사용되고 있습니다. 이는 여러 자연어처리 분야에서 기존의 텍스트 표현보다 높은 성능을 내고 있기 때문입니다.

따라서 이번 시간에는 이들 중 세 번째, 분산 추론 기반 임베딩에 대해서 살펴보겠습니다. (네 번째 기법은 다음 시간에 살펴보겠습니다.)

#### Distributional hypothesis

각각에 대해 자세히 알아보기 앞서 대부분의 단어 표현법에서 사용되는 가정인 distributional hypothesis에 대해서 먼저 살펴보겠습니다.

이 가정은 1957년 Firth에 의해 나타난 것으로 한 마디로 이렇게 표현할 수 있습니다.

`우리는 한 단어를 알기 위해 그 맥락을 유지하는 다른 단어들로부터 알아낼 수 있다.`

예를 들어 보겠습니다. 

우리는 `테스귀노`라는 단어의 뜻을 알고 싶습니다. 그 뜻은 모르지만 우리는 아래 총 4개의 문장에서 단어가 사용됨을 알고 있습니다.

1. 테이블 위에 한 병의 ___ 이/가 있다.
2. 모두들 ___ 을/를 좋아한다.
3. ___ 을/를 운전하기 전에 손대지 마라.
4. 우리는 ___ 을/를 옥수수로부터 만든다.

이제 우리는 저 빈 칸에 적합한 우리가 아는 단어들을 살펴보겠습니다.

| &nbsp; | 1 | 2 | 3 | 4 |
| - | - | - | - | - |
테스귀노 | o | o | o | o |
소음 | x | x | x | x |
휘발유 | o | x | x | x |
콘치즈 | x | o | x | o |
와인 | o | o | o | x |

위의 테이블에서 `o`는 적합함을, `x`는 부적합함을 뜻합니다.

결과들을 놓고 보면 우리는 테스귀노라는 것은 와인과 가장 비슷하며 콘치즈는 그 다음이고 소음은 완전히 다르다는 것을 유추할 수 있습니다.

이처럼 단어에 대한 이해를 하기 위해 사전이 아닌 단어가 쓰이는 맥락을 유지하는 다른 단어들로부터 알 수 있다고 가정하는 것이 바로 distributional hypothesis입니다.

번외로, <a href="https://en.wikipedia.org/wiki/Tesg%C3%BCino">테스귀노(Tesgüino)</a>는 아즈텍 사람들이 만든 옥수수 맥주입니다.

### 2. Word2vec

이번 파트에서는 분산 추론 기반 임베딩 중 Word2vec에 대해 살펴보겠습니다.

Word2vec은 이름 그대로 단어를 벡터로 표현한 것입니다.

이러한 벡터의 표현법은 기존에도 존재하였지만 가장 큰 차이는 세는 방법이 아닌 추론(prediction/classification)에 기반을 둔 방법이라는 점입니다.

더하여 이것은 앞서 살펴본 distributional hypothesis를 사용합니다. 다만, 맥락(context)을 표현하기 위해 문장 전체를 사용하지 않고 일부분만을 사용한다는 것이 차이점입니다.

아래 예시와 함께 살펴보겠습니다.

<img src="img/word2vec1.jpg" width="50%" height="50%" title="word2vec1" alt="word2vec1"></img>

위 그림에서 문장은 `the cat sat on the mat`입니다. 이 문장을 통해 word2vec 학습을 진행합니다.

word2vec의 분류기(classifier)는 하나의 단어를 입력으로 받아 그 주위에 있는 단어들을 추론합니다. 이 때 `주위`의 범위는 우리가 지정할 수 있는데 대체로 5의 크기를 가지도록 합니다. 그림에서는 파란색 박스로 표현했습니다.

따라서 위의 그림에서 분류기는 `the`를 입력으로 받아 문장의 시작을 알리는 `<start>`라는 단어들과 그 뒤에 있는 `cat`, `sat`을 출력으로 내놓아야합니다.

이러한 작업을 문장을 따라가며 진행합니다.

<img src="img/word2vec.gif" width="50%" height="50%" title="word2vec" alt="word2vec"></img>

위 그림에서처럼 가운데 주어지는 단어를 기준으로 양 옆에 있는 단어들을 출력하도록 word2vec 분류기를 학습시킵니다.

이렇게 학습을 시키면 어떤 효과가 있을까요?

<img src="img/word2vec10.jpg" width="50%" height="50%" title="word2vec" alt="word2vec"></img>

이와 같이 `the`라는 단어가 나타났을 때 분류기가 내놓아야하는 출력이 두 가지가 있습니다.

앞서 살펴본 distributional hypothesis을 생각해봅시다. 

word2vec 분류기가 `the`라는 단어를 사전을 보지 못해 모르지만 cat과 mat 앞에 놓여진다는 것을 알 수 있습니다.

이를 통해 해당 단어가 cat, mat과 같은 명사 앞에 높여지는 것을 학습할 수 있습니다.

이처럼 word2vec은 단어들 각각에 대해 추론을 함으로써 그 단어에 뜻과 사용법을 학습하게 되는 것입니다.

<img src="img/word2vec_skip_cbow.jpg" width="50%" height="50%" title="word2vec_skip_cbow" alt="word2vec_skip_cbow"></img>

앞서 살펴본 방법은 word2vec 중 skip-gram이라는 방법입니다. 가운데 단어를 입력으로 주고 주위 단어를 학습시키는 방법이지요.

역으로 주위 단어를 주고 가운데 단어를 맞추도록 학습시킬 수 있습니다. 이를 CBOW라고 불립니다.

<img src="img/word2vec_example.png" width="50%" height="50%" title="word2vec_example" alt="word2vec_example"></img>

출처: https://medium.com/analytics-vidhya/implementing-word2vec-in-tensorflow-44f93cf2665f

이렇게 만든 단어 표현인 word2vec은 또 하나의 강력한 기능이 있습니다.

바로 단어들간의 관계를 찾아낼 수 있다는 것입니다.

위의 그림에서 보시면 단어는 하나의 벡터로 표현되는데 그 벡터들 간의 단순 덧셈/뺄셈 연산으로 단어들 간의 관계를 알 수 있습니다.

예를 들어 king - man + woman 이라는 벡터 연산을 수행하면 그 결과 벡터의 근처에 queen 단어 벡터가 있는 것입니다.

이처럼 우리는 word2vec 분류기가 글 속의 단어들을 추론하도록 학습시켰는데 단어안에 숨겨진 뜻과 사용법까지 학습시킬 수 있습니다.

이것이 word2vec이 인기가 있는 이유입니다.

### 3. fastText

이번 파트에서는 word2vec의 한계점을 극복하기위해 제안된 fastText에 대해 살펴보겠습니다.

word2vec은 상당히 좋은 방법이지만 한계점도 있습니다.

크게 두 가지를 볼 수 있습니다.

- 단어 토큰을 만드는 것은 어려운 작업이다
- 비슷하지만 다른 형태의 단어들이 많다

**첫 번째 한게점**에 대해서 살펴보겠습니다.

영어의 경우 단어 토큰을 만드는 것은 쉽습니다. 대체로 띄어쓰기를 기준으로 쪼개면 되니까요.

하지만 다른 언어들은 그렇지 않은 경우가 많습니다.

<img src="img/turkish.jpg" width="50%" height="50%" title="turkish" alt="turkish"></img>

위 그림은 터키어와 그 번역입니다. 왼쪽에 적혀진 터키어 글은 띄어쓰기로 쪼개기 어렵습니다. 그렇다면 따로 토큰을 만드는 프로그램이 필요한데 이 역시 힘든 일입니다.



**두 번째 한계점**은 지난 시간에 보았고 여러분도 익숙하실 것입니다.

한국어의 경우 동사의 형태가 매우 다양합니다. 이를 서로 다른 단어라고 취급하면 단어에 대한 제대로 된 학습이 어려울 것입니다.

이를 해결하기 위해 우리는 지난 시간에 어간 추출(stemming) 전처리 기술에 대해 배웠습니다.

하지만 그 기술도 추가적으로 필요한 것이고 여기에서 발생하는 오류가 단어 학습에 있어서도 영향을 준다면 문제가 됩니다.


#### fastText
이러한 한계점을 극복하기 위해 제안된 것이 fastText입니다.

fastText는 단어 표현이 아니라 그보다 작은 단위(subword)의 표현을 만들었습니다.

단어보다 작은 단위는 글자가 있을 수 있죠. 하지만 글자 하나만을 본 것이 아니라 글자를 뭉쳐서 보았습니다.

<img src="img/fasttext_lose.jpg" width="50%" height="50%" title="fasttext_lose" alt="fasttext_lose"></img>

위의 그림은 `lose`라는 단어를 글자 단위로 쪼개어 살펴본 것입니다.

먼저 해당 단어의 앞뒤에 특수 문자(<와 >)을 두어 단어의 앞뒤를 표현합니다.

그 후 글자 길이만큼을 쪼갭니다.

예를 들어 글자 길이를 2라고 할 때 `<l`, `lo`, `os`, `se`, `e>`와 같은 subword를 만들 수 있습니다.

같은 방법으로 길이를 3일 때, 4일 때로 정한 후 마지막에 원래 단어도 넣습니다.

이렇게 다양하게 단어를 글자 길이에 맞춰 쪼갠 것을 각각의 벡터로 만들어 이를 한꺼번에 학습시키는 것이 fastText라고 볼 수 있습니다.

참고로 여기서 n-gram이라는 개념을 얘기합니다.

간단히 n-gram이란 시계열(sequence) 형태의 것을 길이에 따라 쪼개는 방법으로 이 때 쪼개어진 것의 개수가 최대가 되도록 하나씩 움직이며 쪼개는 것을 뜻합니다.

위의 예시처럼 2-gram인 경우 길이 2의 글자를 가지도록 하며 처음부터 하나씩 움직이며 만드는 것을 알 수 있습니다.

이 방법은 시계열 데이터에 적용되고 그 단위 역시 다양하게 할 수 있습니다.

<img src="img/fasttext_loze.jpg" width="50%" height="50%" title="fasttext_loze" alt="fasttext_loze"></img>

이러한 식으로 fastText는 단어를 표현하게 되는데 이 경우 오탈자와 처음 보는 단어(out of vocabulary)에 강력하게 동작합니다.

예를 들어 위의 그림에서처럼 `loze`라는 단어를 보게 되었을 때 이를 subword 단위로 쪼개어 보게 됩니다.

그럼 기존의 단어인 `lose`와 겹치는 부분이 많이 있기에 이를 통해 `loze`는 `lose`의 오탈자임을 유추할 수 있습니다.

<img src="img/fasttext_korean.png" width="50%" height="50%" title="fasttext_korean" alt="fasttext_korean"></img>

출처: KIM, Seonhghyun, et al. Korean-Optimized Word Representations for Out-of-Vocabulary Problems Caused by Misspelling Using Sub-character Information. In: Future of Information and Communication Conference. Springer, Cham, 2019. p. 25-32.

비슷한 방법으로 처음 보는 단어의 경우 그와 유사한 단어들을 찾아낼 수 있습니다.

위의 그림에서 학습된 fastText는 한국어의 특징인 자모 단위로 쪼개어 학습한 것으로 `페날티`라는 단어를 본 적이 없음에도 그와 유사한 단어들인 `페널티`를 찾을 수 있게 되는 것입니다.

이처럼 텍스트를 표현하는 방법에는 여러 가지가 있으며 이번 시간에는 그 중 분산 추론 기반 임베딩인 word2vec과 fastText에 대해서 살펴보았습니다.

다음 시간에는 또 다른 방법인 분산 문맥 임베딩에 대해서 살펴보도록 하겠습니다.

자신이 다룰 텍스트 데이터 및 문제가 어떤 특징을 가지는지에 따라 적합한 텍스트 표현을 사용하시기를 권장합니다.

---

<span style="color:rgb(120, 120, 120)">본 학습 자료를 포함한 사이트 내 모든 자료의 저작권은 엘리스에 있으며 외부로의 무단 복제, 배포 및 전송을 불허합니다.

Copyright @ elice all rights reserved</span>